# 오늘 실습은 인공데이터가 아닌 실제데이터(mnist)로 딥러닝 구현!

### 이건 실행시켜야 하나........?

In [1]:
#-- 리스트 8-1-(1)
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data() 
# x_train:입력벡터. 필기체 숫자 이미지 10만개 / y_train : 정수형 데이터(한 이미지 당 한 정수)
# x_test / y_test  ==> ?

Using TensorFlow backend.


In [2]:
import numpy as np
np.savez('mnist.npz',x_train=x_train, y_train=y_train,x_test=x_test,y_test=y_test)

In [ ]:
%reset

## 여기서부터 실행시키면 됨!

In [ ]:
import numpy as np
outfile = np.load('mnist.npz')
x_train=outfile['x_train']
y_train=outfile['y_train']
x_test=outfile['x_test']
y_test=outfile['y_test']
y_test_origin = y_test.copy()

In [ ]:
# - 리스트 8-1-(2)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(1, figsize=(12, 3.2))
plt.subplots_adjust(wspace=0.5)
plt.gray()
for id in range(3):
    plt.subplot(1, 3, id + 1)
    img = x_train[id, :, :]  # 이미지 크기 : 28x28 / id : 각 이미지의 index. / : , : ==> 이미지(행렬구조)... 28x28모두 불러오겠다!
    plt.pcolor(255 - img)  # 이미지가 그냥 5로만 보이지만,실제로는 5부분에 255에 가까운,흰 바탕 부분에 0에 가까운 값이 있음!==>반전시키기!
    plt.text(18, 26, "L:%d" % y_train[id],  #y_train : 목적벡터(라벨값) /각 이미지에 대해서 레이블링(라벨) 되어있음 ==> 그래프 하단에 출력시키기
             color='cornflowerblue', fontsize=14)
    plt.xlim(0, 27)
    plt.ylim(27, 0)
    plt.grid('on', color='white')
plt.show()

In [ ]:
#-- 리스트 8-1-(3)
from keras.utils import np_utils


x_train = x_train.reshape(60000, 784) # (A) 입력벡터가 열벡터로서 표현되도록, 입력으로 들어온 2차원 이미지데이터를 열벡터로 모양 바꾸기(6만개의 데이터를 28*28=784개의 노드 가진 세로 열벡터로!!)
x_train = x_train.astype('float32') # (B) astype함수 호출시켜서 int(정수형) 타입을 float(실수형)으로 바꾸기
x_train = x_train / 255 # (C) 255로 나누면 0~1사이의 값으로 정규화됨(normalization)
num_classes = 10
y_train = np_utils.to_categorical(y_train, num_classes) # (D) 원핫코드로 바꾸기!(~~의 to_categorical 사용하면댐) ==> 10개의 출력(0000 or 1)으로 바꾸되, 원핫코드로 바꿔주기!


x_test = x_test.reshape(10000, 784) # test set은 10000개 (아까 위에서 훈련집합은 60000개)
x_test = x_test.astype('float32')
x_test = x_test / 255
y_test = np_utils.to_categorical(y_test, num_classes)

In [ ]:
#-- 리스트 8-1-(4) ==> p.326의 그림 8-2 : 2층 MLP 구현하기
np.random.seed(1)
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam


model = Sequential() # (A) 직렬구조의 네트워크...쓸때... Sequential 이라는 객체 써야함(객체생성)
model.add(Dense(16, input_dim=784, activation='sigmoid')) # (B)은닉층 (노드)수?를 16개로, 은닉층의 활성함수로 시그모이드 쓰기 / ***Dense(200)으로 바꿔보기! ==> 무작정 늘린다고 성능 좋아지는 거 아니다~

#model.add(Dense(12, activation='sigmoid')) # *** 쉅시간에 네트워크 구조 하나 추가해봄!! 네트워크 바꿔보기!! ==> 좀 시간걸리겟지..훈련시켜보기 ==> 오히려 성능(Test Accuracy) 떨어짐..!*** 

model.add(Dense(10, activation='softmax')) # (C) 출력층 노드 수를 10개로, 출력층의 활성함수로 소프트맥스 쓰기

model.compile(loss='categorical_crossentropy',  # 교차엔트로피로 쓰겠다(비용함수??) / compile함수를 이용해서 .....정의???ㅠㅠ
optimizer=Adam(), metrics=['accuracy']) # (D)훈련이 잘 되었는지를 측정하는 measure로서 accuracy쓰겠다.

### 여기까지는 기본 셋팅한 것. (구성요소들 정의)

In [ ]:
#-- 리스트 8-1-(5)
import time


startTime = time.time()
history = model.fit(x_train, y_train, epochs=10, batch_size=1000,  #(트레이닝에 쓰이는 입력벡터, 목적벡터, 각각의 트레이닝 데이터에 대해서 10번 돌리겠다(순회), 
                    verbose=1, validation_data=(x_test, y_test)) # (A)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print("Computation time:{0:.3f} sec".format(time.time() - startTime)) # 실제 수행 시간 알 수 있음.

#결과보면 100개 중에서 89개 맞췄다는 걸 알수있음.
#Computation time : 약 3초정도밖에 안걸림

In [ ]:
#-- 리스트 8-1-(6)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


plt.figure(1, figsize=(10, 4))
plt.subplots_adjust(wspace=0.5)


plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='training', color='black')
plt.plot(history.history['val_loss'], label='test',
         color='cornflowerblue')
plt.ylim(0, 10)
plt.legend()
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')


plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='training', color='black')
plt.plot(history.history['val_accuracy'],label='test', color='cornflowerblue')
plt.ylim(0, 1)
plt.legend()
plt.grid()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.show()

# 첫번째 : 검정선은 loss함수의 에러값 / 파란색은 test 데이터의 검증값  ==>  둘다 0으로 수렴하고 있으니까 잘 훈련되고 있다는 걸 알 수 있음.(오버피팅 안일어남)
# 두번째 그래프도 마찬가지!

In [ ]:
#-- 리스트 8-1-(7)
def show_prediction():
    n_show = 96
    y = model.predict(x_test) # (A)
    plt.figure(2, figsize=(12, 8))
    plt.gray()
    for i in range(n_show):
        plt.subplot(8, 12, i + 1)
        x = x_test[i, :]  # 아까 위에서 데이터들을 다 직렬화시켰음(열벡터로) ==> 이거를 다시 28*28 이미지 형태로 바꿔주는 것.
        x = x.reshape(28, 28) # 28*28 크기의 이미지 형태로 다시 변환시키기~
        plt.pcolor(1 - x)  # 반전시키기. 0~1사이의 값을 x가 갖기 때문에 1에서 빼주면... 반전된 값.
        wk = y[i, :]
        prediction = np.argmax(wk)
        plt.text(18, 25.5,"%d" % y_test_origin[i],fontsize=12) # y_test_origin[i] : 맨위에서 y_test.copy() 헀었음. 여기에서 읽어들이는거..
        plt.text(22, 25.5,"%d" % prediction,fontsize=12)
        if prediction != np.argmax(y_test[i, :]):
            plt.plot([0, 27], [1, 1], color='cornflowerblue', linewidth=5)
        plt.xlim(0, 27)
        plt.ylim(27, 0)
        plt.xticks([], "")
        plt.yticks([], "")
#-- 메인
show_prediction()
plt.show()


# 이정도면 훌륭!! 대부분 잘 분류함. 근데 요즘에는 거의 99.9% 잘 분류해서 이정도가지고는 실용화하지는 못함.

In [ ]:
#-- 리스트 8-1-(8)
np.random.seed(1)
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam


model = Sequential()
model.add(Dense(16, input_dim=784, activation='relu')) # (A)활성함수로 렐루함수로 쓰기(이거 하나만 바꿨는데도 엄청난 효과가!!)
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(), metrics=['accuracy'])


startTime = time.time()
history = model.fit(x_train, y_train, batch_size=1000, epochs=10,
                    verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print("Computation time:{0:.3f} sec".format(time.time() - startTime))

In [ ]:
# 리스트 8-1-(9)
show_prediction()
plt.show()

In [ ]:
#-- 리스트 8-1-(10)
# 1층째의 무게 시각화
w = model.layers[0].get_weights()[0] # layers[0]이면 첫번째 히든레이어(은닉층), get_weights()[0] 에서 0이아니라 1이면 바이어스 노드까지..?포함한 가중치..?
plt.figure(1, figsize=(12, 3))
plt.gray()
plt.subplots_adjust(wspace=0.35, hspace=0.5)
for i in range(16):
    plt.subplot(2, 8, i + 1)
    w1 = w[:, i] # 하나의 히든노드에 연결되어있는 입력노드의 가중치들. 총 784개(28*28)==입력노드의 차원. 열벡터!
    w1 = w1.reshape(28, 28) # 열벡터를 28*28의 형태로 변환시키기~
    plt.pcolor(-w1)
    plt.xlim(0, 27)
    plt.ylim(27, 0)
    plt.xticks([], "")
    plt.yticks([], "")
    plt.title("%d" % i)
plt.show()

In [ ]:
%reset

In [ ]:
#-- 리스트 8-2-(1)
import numpy as np
# from keras.datasets import mnist
from keras.utils import np_utils
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

outfile = np.load('mnist.npz')
x_train=outfile['x_train']
y_train=outfile['y_train']
x_test=outfile['x_test']
y_test=outfile['y_test']
y_test_origin = y_test.copy()


x_train = x_train.reshape(60000, 28, 28, 1) # 3차원 *텐서*로 이미지를 표현! 28*28크기의 이미지가 1개 있는 것. 그런게 6만개 있음! 이미지!
x_test = x_test.reshape(10000, 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255  # 0~1사이의 값을 갖도록 정규화
x_test /= 255
num_classes = 10
y_train = np_utils.to_categorical(y_train, num_classes) # 목적벡터를 원핫코드 형식으로 바꿔주기
y_test = np_utils.to_categorical(y_test, num_classes)

In [ ]:
1/9

In [ ]:
#-- 리스트 8-2-(2)
import matplotlib.pyplot as plt
%matplotlib inline


id_img = 2

myfil1 = np.array([[1/9, 1/9, 1/9],
                   [1/9, 1/9, 1/9],
                   [1/9, 1/9, 1/9]], dtype=float) # (A) average filter : 각 요소들을 9분의1씩 해주는 것 == 평균내는 것 == **블러링 효과**
myfil2 = np.array([[0, -1, 0],
                   [-1, 5, 1],
                   [0, -1, 0]], dtype=float) # (B) sharping filter : 주변보다 현재 가운데 픽셀을 두드러지게 하는 것 == **샤프닝 효과**

#myfil1 = np.array([[-1, -1, -1],
#                   [0, 0, 0],
#                   [1, 1, 1]], dtype=float) # (A) horizontal edge filter : 교수님이 책과는 좀 다르게, 좀 더 일반적인 커널! ==> **수평~**

# myfil2 = np.array([[-1, 0, 1],
#                   [-1, 0, 1],
#                   [-1, 0, 1]], dtype=float) # (B) vertical edge filter : **수직~**


x_img = x_train[id_img, :, :, 0]
img_h = 28
img_w = 28
x_img = x_img.reshape(img_h, img_w)  # 28*28 2차원 행렬로 입력 이미지를 가져옴..?
out_img1 = np.zeros_like(x_img)  # 이미지 각각에 대해서 출력값을 저장. 원래 크기인 28*28 형태로..?
out_img2 = np.zeros_like(x_img)

# 필터 처리
for ih in range(img_h - 3):
    for iw in range(img_w - 3):
        img_part = x_img[ih:ih + 3, iw:iw + 3]  # 이미지의 일부(커널만큼) 떼어오는 것 ==> 입력으로ㄱㄱ해서 컨볼루션 / 텐서 구조로다가 이미지를 변환시켜놨었음. / x_img는 이미지 자체(2차원) / ih:ih+3  ==> 만약 ih=0이면 0,1,2 까지만! 이미지에서 뗴어오기!
        out_img1[ih + 1, iw + 1] = \
            np.dot(img_part.reshape(-1), myfil1.reshape(-1))# 내적시켜서 output img(out_img)에 저장하는거랑 같음! (그냥 행렬 요소 곱 하는거랑...)
        out_img2[ih + 1, iw + 1] = \
            np.dot(img_part.reshape(-1), myfil2.reshape(-1)) 


# - 표시
plt.figure(1, figsize=(12, 3.2))
plt.subplots_adjust(wspace=0.5)
plt.gray()
plt.subplot(1, 3, 1)
plt.pcolor(1 - x_img)
plt.xlim(-1, 29)
plt.ylim(29, -1)
plt.subplot(1, 3, 2)
plt.pcolor(-out_img1)
plt.xlim(-1, 29)
plt.ylim(29, -1)
plt.subplot(1, 3, 3)
plt.pcolor(-out_img2)
plt.xlim(-1, 29)
plt.ylim(29, -1)
plt.show()


# 두번째 : 블러링 효과
# 세번째 : 샤프닝 효과 / 바깥보녀 0덧대기(zero-padding)안해서 이미지 사이즈가 줄음!(테두리부분 사라짐)

In [ ]:
#-- 리스트 8-2-(3)  ==>  케라스를 쓰면 네트워크를 구성하는게 직관적. 쉬움.
import numpy as np
np.random.seed(1)
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
import time


model = Sequential()
model.add(Conv2D(8, (3, 3), padding='same',  # 층하나 쌓은 것!(add). 컨볼루션 함수. 3x3짜리 커널, padding:입력특징맵의 사이즈와 달라지지 않도록 제로패딩하겠다, 
                 input_shape=(28, 28, 1), activation='relu')) # (A)input_shape : 입력이미지사이즈(3차원행렬), activation:활성함수
model.add(Flatten()) # (B) 직렬화 하는 과정! 층을 하나 만든다기 보다는 변환하는 것. 마지막에 분류 위한 f-c(완전연결층)에 연결하기 위해서!!
model.add(Dense(10, activation='softmax'))  #Dense함수(f-c),  : 10개노드(0~9필기체 숫자 인식하는거라서)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),  # optimizer : Adam 알고리즘(이번주이론시간에배움)
              metrics=['accuracy']) # accuracy : 참,거짓 판별한 것 중 참으로 판별한 것의 개수의 비율. 이걸로 성능 평가함.(성능평가기준 : accuracy, recall) 
startTime = time.time()
history = model.fit(x_train, y_train, batch_size=1000, epochs=20,  # fit함수 : training하는 과정. (입력벡터, 목적벡터, 배치사이즈1000(미니배치), 에포크 20세대,)
                    verbose=1, validation_data=(x_test, y_test)) #verbose :????, 검증할 목적으로 테스트데이터집합 씀.
score = model.evaluate(x_test, y_test, verbose=0)  # evaluate : 마지막에 성능 확인하는 작업.
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print("Computation time:{0:.3f} sec".format(time.time() - startTime))

# 결과 ==> Test accuracy: 0.9707 ==> DMLP로 했을 때 0.95? 94? 였는데, CNN쓰면 정확도가 훨씬 올라감! (CNN만 썼을 때...)

In [ ]:
#-- 리스트 8-1-(7)
def show_prediction():
    n_show = 96
    y = model.predict(x_test) # (A)
    plt.figure(2, figsize=(12, 8))
    plt.gray()
    for i in range(n_show):
        plt.subplot(8, 12, i + 1)
        x = x_test[i, :]
        x = x.reshape(28, 28)
        plt.pcolor(1 - x)
        wk = y[i, :]
        prediction = np.argmax(wk)
        plt.text(22, 25.5, "%d" % prediction, fontsize=12)
        if prediction != np.argmax(y_test[i, :]):
            plt.plot([0, 27], [1, 1], color='cornflowerblue', linewidth=5)
        plt.xlim(0, 27)
        plt.ylim(27, 0)
        plt.xticks([], "")
        plt.yticks([], "")

In [ ]:
# 리스트 8-2-(4)
show_prediction()
plt.show()

# 전체에서 틀린게 2개밖에 없음!

In [ ]:
# 리스트 8-2-(5)
plt.figure(1, figsize=(12, 2.5))
plt.gray()
plt.subplots_adjust(wspace=0.2, hspace=0.2)
plt.subplot(2, 9, 10)
id_img = 12
x_img = x_test[id_img, :, :, 0]
img_h = 28
img_w = 28
x_img = x_img.reshape(img_h, img_w)
plt.pcolor(-x_img)
plt.xlim(0, img_h)
plt.ylim(img_w, 0)
plt.xticks([], "")
plt.yticks([], "")
plt.title("Original")


w = model.layers[0].get_weights()[0] # (A)
max_w = np.max(w)
min_w = np.min(w)
for i in range(8):
    plt.subplot(2, 9, i + 2)
    w1 = w[:, :, 0, i]
    w1 = w1.reshape(3, 3)
    plt.pcolor(-w1, vmin=min_w, vmax=max_w)
    plt.xlim(0, 3)
    plt.ylim(3, 0)
    plt.xticks([], "")
    plt.yticks([], "")
    plt.title("%d" % i)
    plt.subplot(2, 9, i + 11)
    out_img = np.zeros_like(x_img)
    # 필터 처리
    for ih in range(img_h - 3):
        for iw in range(img_w - 3):
            img_part = x_img[ih:ih + 3, iw:iw + 3]
            out_img[ih + 1, iw + 1] = \
            np.dot(img_part.reshape(-1), w1.reshape(-1))
    plt.pcolor(-out_img)
    plt.xlim(0, img_w)
    plt.ylim(img_h, 0)
    plt.xticks([], "")
    plt.yticks([], "")
plt.show()

# 첫번째 행 : 8개의 커널의 값들을 이미지로써 보여준 것! 검은색이 높은값(큰값), 흰색이 낮은값(작은값..0쪽)
# ==> 이런 커널들을 사람이 만드는 것이 아니라, 네트워크가 자동적으로 생성한 것. 과거에는 이렇게 학습을 통해서가 아니라 인간이 직접 찾음..

# 두번째 행 : 입력한 원본이미지(9)에 각 커널들을 적용했을 때 나타나는 이미지. 8개의 특징맵

In [ ]:
# 리스트 8-2-(6) ==> 몇 줄 안되지만, 네트워크 구현하고 학습까지 수행한 코드!
import numpy as np
np.random.seed(1)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
import time


model = Sequential()
model.add(Conv2D(16, (3, 3),
                 input_shape=(28, 28, 1), activation='relu')) #1번째 층 : 컨볼루션 층. 3x3짜리 커널 16개쓰겠다, 입력벡터의 크기(28x28x1..텐서구조 쓰기위해), 활성함수로 렐루함수
model.add(Conv2D(32, (3, 3), activation='relu')) #2번째 층 : 컨볼루션 층. 3x3짜리 커널 32개 쓰겠다.
model.add(MaxPooling2D(pool_size=(2, 2))) # (A) 3번째 층 : 최대풀링 층
model.add(Conv2D(64, (3, 3), activation='relu')) #4번째 층 : 컨볼루션 층

model.add(MaxPooling2D(pool_size=(2, 2))) # (B)5번째 층 : 최대풀링 층
model.add(Dropout(0.25)) # (C) 그다음에 드롭아웃 적용(규제) ==> 중간층 노드들의 p=0.25만큼만 가지고 학습시키겠다!

model.add(Flatten()) # f-c로 넘어가기 전에 열벡터로 쭉 펴는 플랫튼 과정 필요.
model.add(Dense(128, activation='relu')) # Dense함수..??
model.add(Dropout(0.25)) # (D) 다시한번 규제. p=0.25
model.add(Dense(num_classes, activation='softmax')) # num_classes = 10 ==> 출력 노드를 10개로 정해줌. 마지막 활성함수는 소프트맥스로 쓰겠다


model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy']) # 목적(비용)함수로 교차엔트로피쓰겠다, Adam이라는 알고리즘쓰겠다, 성능 측정 판단 기준은 accuracy 쓰겠다


startTime = time.time()


history = model.fit(x_train, y_train, batch_size=1000, epochs=20, 
                    verbose=1, validation_data=(x_test, y_test)) # fit함수 써서 실제로 트레이닝.학습. 배치사이즈, 에포크,,, 중간과정(verbose?)


score = model.evaluate(x_test, y_test, verbose=0) # 마지막에 검증하는 단계
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print("Computation time:{0:.3f} sec".format(time.time() - startTime))

# 아래 결과들 Epoch(에포크) 마다 각각 봐보면, 한 에포크 당 40초정도 걸림... 20에포크 다하면 한 800초(10여분 걸림..)
# 점점 acc(accuracy) 올라감

In [ ]:
# 리스트 8-2-(7)
show_prediction()
plt.show()